# Analise do dataset de cachorros

Nesse notebook vamos efetuar uma análise exploratória do dataset de cachorros a fim de entender as informações que lá existem e procurar algum padrão ou obter alguns insigths sobre o contexto.

No final vamos procurar criar um modelo que possa prever a raça do cachorro de acordo com as features levantadas.

Como premiça nosso modelo deve ter pelo menos 70% de acurácia e precisão.

## Importação do dataset

Inicialmente vamos importar o dataset, utilizando o pandas e vamos visulizar as informações através da biblioteca de gráficos Seaborn e Matplotlib.

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

sns.set_theme()

In [ ]:
df = pd.read_json("cachorros.json")
df.head()

## Análise incial do dataset

Vamos buscar visualizar como os dados estão categorizados, distribuídos e correlacionados.

In [ ]:
df.info()

In [ ]:
df.Sex = df.Sex.astype('category')

In [ ]:
df.info()

In [ ]:
df.Sex

In [ ]:
df.describe()

### Histograma

No histograma podemos visualizar a distribuição dos dados de acordo com a frequência que aparecem.

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12,8))

plt.suptitle('Distribuições')

sns.histplot(x='Age', data=df, kde=True, ax=ax[0][0]).set(title='Distribuição da idade', xlabel='Idade', ylabel='Quantidade')
sns.histplot(x='Weight', data=df, kde=True, ax=ax[0][1]).set(title='Distribuição do peso', xlabel='Peso', ylabel='Quantidade')
sns.histplot(x='Height', data=df, kde=True, ax=ax[1][0]).set(title='Distribuição do altura', xlabel='Altura', ylabel='Quantidade')

plt.tight_layout()
plt.show()

### Boxplot

O boxplot nos permitem visualizar como os dados estão distribuidos dentro dos quartiles e se existem outliers.

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12,8))

plt.suptitle('Outliers')

sns.boxplot(x='Age', data=df, ax=ax[0][0]).set(title='Outlier para Idade', xlabel='Idade')
sns.boxplot(x='Weight', data=df, ax=ax[0][1]).set(title='Outlier para Peso', xlabel='Peso')
sns.boxplot(x='Height', data=df, ax=ax[1][0]).set(title='Outlier para Altura', xlabel='Altura')

plt.tight_layout()
plt.show()

### Correlação 

Podemos agora identificar informações que possam ser correlacionadas. Por correlacionadas podemos entender que dada uma variável X outra variavel Y pode aumentar ou diminuir na mesma medida que X aumenta ou diminui.

Vale resaltar que correlação não implica em causualidade.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(18, 10))

plt.suptitle("Correlação entre variáveis")

corr = df.corr()
sns.heatmap(corr, ax=ax1).set(title='Correlação')

sns.scatterplot(x='Weight', y='Height', hue='Breed', data=df, ax=ax2).set(title='Gráfico de dispersão x Raça', xlabel='Peso', ylabel='Altura')
sns.set(font_scale=1.4)

sns.scatterplot(x='Weight', y='Height', hue='Sex', data=df, ax=ax3).set(title='Gráfico de dispersão x Sexo', xlabel='Peso', ylabel='Altura')
sns.set(font_scale=1.4)

plt.tight_layout()
plt.show()

#### Criação de coluna - Feature Engineering

In [ ]:
df['Category'] = pd.cut(df['Weight'], bins=[0, 15, 30, 100], labels=['small', 'medium', 'big'])
df.head()

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x='Category', data=df).set(title='Quantidade de cachorros por porte', xlabel='Categoria', ylabel='Quantidade')
plt.tight_layout()
plt.show()

## Analise de variáveis categoricas

Vamos agora analisar as variaveis categoricas, ou seja, variáveis que são expressas por textos, podendo elas serem ordinais ou nominais.

### Cores

In [ ]:
plt.figure(figsize=(18,10))
sns.countplot(data=df, y='Color', order=df.Color.value_counts().index).set(title='Quantidade de animais por cor', ylabel='Cor', xlabel='Quantidade', xticks=np.arange(0, 130, 10))
plt.tight_layout()
plt.show()

In [ ]:
len(df.Color.value_counts().keys())

#### Transformação de colunas - Feature Engineering

Vamos transformar nossa única coluna que contem as cores dos cachorros em várias colunas booleanas (Verdadeiro/Falso), com todas as cores, assinalando quais cores aquele animal possue. 

In [ ]:
d = pd.DataFrame(columns=['Cores'])
d = d.append({'Cores': ['Azul']}, ignore_index=True)
d = d.append({'Cores': ['Verde']}, ignore_index=True)
d = d.append({'Cores': ['Verde', 'Azul']}, ignore_index=True)
d

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

label = MultiLabelBinarizer()
label.fit_transform(d['Cores'].values)

In [ ]:
pd.DataFrame(label.fit_transform(d['Cores'].values), columns=label.classes_)

##### Tratamento dos valores

Antes de iniciar o processo de criação de novas colunas, primeiro precisamos fazer um tratamento nos dados existentes, normalizando-os. 

* Remover caracteres especiais

* Converter a palavra em um vetor

* Traduzir as cores

In [ ]:
dicionario = {
    'abricot': 'damasco',
    'tan': 'bronze',
    'fawn': 'fulvo',
    'rust': 'ferrugem',
    'red': 'vermelho',
    'blue': 'azul',
    'gray': 'cinza',
    'white': 'branco',
    'casbronzeadoho': 'castanho',
    'sable': 'areia'
}

def traduzir_palavras(text: str):
    
    for chave, valor in dicionario.items():
       text = text.replace(chave, valor)
    
    return text

In [ ]:
def split_colors(text: str):
    text = ','.join(text.split(' e '))
    text = ','.join(text.split(' & '))
    text = ','.join(text.split(' - ')).lower()
    text = traduzir_palavras(text)
    return text.split(',')

In [ ]:
df['Colors'] = df.Color.apply(split_colors)

In [ ]:
label = MultiLabelBinarizer()
colors = label.fit_transform(df['Colors'])
df_colors = pd.DataFrame(colors, columns=label.classes_)
df_colors.columns

In [ ]:
len(df_colors.columns)

In [ ]:
df = pd.concat([df, df_colors], axis=1)
df.head()

#### Personalidade

In [ ]:
plt.figure(figsize=(18,10))
sns.countplot(y='Personality', data=df, order=df.Personality.value_counts().index).set(title='Personalidades', xlabel='Quantidade', ylabel='Personalidade', xticks=np.arange(0, 110, 10))
plt.tight_layout()
plt.show()

## Treinamento de um modelo

Vamos treinar um modelo para prever a raça de um cachorro. 

Primeiramente vamos dividir a base entre treino e teste com o objetivo de termos uma massa para treinamento e outra para vaildação a fim de validar se nosso modelo generaliza bem dados não vistos.

In [ ]:
X = df[df.columns.difference(['Category', 'Color', 'Colors', 'Origin', 'Personality', 'Breed'])]
y = df['Breed']

Vamos utilizar cerca de 75% da base para treinamento e 25% para validação

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix, SCORERS

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
print(X_test.shape)
print(y_test.shape)

#### Modelo Dummy

Vamos criar um modelo Dummy a fim de validar qual será nossa base line.

In [ ]:
dummy = DummyClassifier()
dummy.fit(X_train, y_train)

y_dummy = dummy.predict(X_train)

print(accuracy_score(y_train, y_dummy))
print(precision_score(y_train, y_dummy, average='weighted'))
print(classification_report(y_train, y_dummy))

In [ ]:
pd.DataFrame(confusion_matrix(y_train, y_dummy), columns=dummy.classes_, index=dummy.classes_)

### Treinar um modelo mais inteligênte

In [ ]:
from sklearn.svm import SVC

svc = SVC(random_state=10)
svc.fit(X_train, y_train)

y_pred = svc.predict(X_train)

print(accuracy_score(y_train, y_pred))
print(precision_score(y_train, y_pred, average='weighted'))
print(classification_report(y_train, y_pred))

In [ ]:
pd.DataFrame(confusion_matrix(y_train, y_pred), columns=svc.classes_, index=svc.classes_)

In [ ]:
print(SCORERS.keys())

In [ ]:
result = cross_validate(SVC(), X_train, y_train, cv=10, scoring=('accuracy', 'precision_macro'))
result

In [ ]:
np.mean(result.get('test_accuracy'))

In [ ]:
y_pred = svc.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred, average='weighted'))
print(classification_report(y_test, y_pred))

In [ ]:
pd.DataFrame(confusion_matrix(y_test, y_pred), columns=svc.classes_, index=svc.classes_)